# Raleigh Crime Analysis

This is a demo to perform basic analysis on our Raleigh Crime dataset in R.  In this demo, we will perform basic analysis, particularly around quick visualization.

In [ ]:
install.packages('nlme', repos = "http://cran.us.r-project.org")
install.packages('tidyverse', repos = "http://cran.us.r-project.org", dependencies = TRUE)
install.packages('RODBC', repos = "http://cran.us.r-project.org", dependencies = TRUE)
install.packages('ggplot2', repos = "http://cran.us.r-project.org", dependencies = TRUE)
install.packages('scales', repos = "http://cran.us.r-project.org", dependencies = TRUE)

In [ ]:
library(tidyverse)
library(RODBC)
library(ggplot2)
library(scales)

Connect to a local database and load Raleigh crime data.  Note that we loaded this data set earlier in the talk (the SQL Server portion of the show), so you may need to follow those steps to follow along.

In [ ]:
conn <- odbcDriverConnect("driver={SQL Server};server=LOCALHOST;database=RaleighCrime;trusted_connection=true")
raleighcrime <- sqlQuery(conn,
  "SELECT
	i.BeatID,
	i.IncidentCode,
	ic.IncidentDescription,
	it.IncidentType,
	i.IncidentDate,
	i.IncidentNumber
FROM dbo.Incident i
	INNER JOIN dbo.IncidentCode ic
		ON i.IncidentCode = ic.IncidentCode
	INNER JOIN dbo.IncidentType it
		ON ic.IncidentTypeID = it.IncidentTypeID;"
)

The first step when analyzing a data set:  review the variables and basic summary information.

In [ ]:
str(raleighcrime)

We're going to want to do a bit of cleanup here.  We'll make the text values (including the ill-named Incident Number) into strings and split out date into several columns for easier analysis downstream.

In [ ]:
raleighcrime$IncidentNumber <- as.character(raleighcrime$IncidentNumber)
raleighcrime$IncidentCode <- as.character(raleighcrime$IncidentCode)
raleighcrime$IncidentType <- as.character(raleighcrime$IncidentType)
raleighcrime$IncidentDescription <- as.character(raleighcrime$IncidentDescription)
raleighcrime$IncidentYear <- as.integer(format(raleighcrime$IncidentDate, format="%Y"))
raleighcrime$IncidentMonth <- as.integer(format(raleighcrime$IncidentDate, format="%m"))
raleighcrime$IncidentDay <- as.integer(format(raleighcrime$IncidentDate, format="%d"))

The `complete_cases()` function returns a vector of true/false, where it returns `TRUE` when all variables in an observation have values and `FALSE` if there are NA values.  We will filter out any observations with missing values.

In [ ]:
raleighcrime <- raleighcrime[complete.cases(raleighcrime), ]

We didn't lose any observations, so we had a complete data set to begin with.

In [ ]:
str(raleighcrime)

Next up, we'll create a couple of groups for further analysis.

In [ ]:
raleigh.crime.by.year <- raleighcrime %>%
                            group_by(IncidentYear) %>%
                            summarize(n = n())
raleigh.crime.by.incident.type <- raleighcrime %>%
                                     group_by(IncidentType) %>%
                                     summarize(n = n())

## Scatter Plots

Create a scatter plot of Raleigh incidents by year.

In [ ]:
plot(raleigh.crime.by.year)

We can see that 2014 has fewer incidents than the other years.  This is because 2014 is an incomplete year, so we will need to take that into account when performing analysis.  Ideally, I'm going to skip 2014.

## Histograms

Next up, we'll look at a histogram of incident types.

In [ ]:
hist(raleigh.crime.by.incident.type$n)

This tells us that most of our incident types have between 0 and 10,000 incidents in the data set.  We'll be most interested in the most common incident types, as that gives us a natural and interesting filter.

So let's look at which types are most popular, starting with the most popular groups and then looking at the mid-range groups.

In [ ]:
raleigh.crime.by.incident.type %>% filter(n >= 80000) %>% arrange(desc(n))

In [ ]:
raleigh.crime.by.incident.type %>% filter(n >= 20000, n < 80000) %>% arrange(desc(n))

## Box and Whisker Plots

We will look at counts of incident types on a per-month basis to see the variance for each of the major incident types.  This will help us see if there are certain months which are significantly different from the norm.

In [ ]:
raleigh.crime.by.type.and.month <- raleighcrime %>% 
                                    filter(IncidentYear < 2014) %>% 
                                    group_by(IncidentType, IncidentYear, IncidentMonth) %>% 
                                    summarize(n = n())
popular.crimes <- raleigh.crime.by.incident.type %>% filter(n >= 20000)

What we have are two sets of data:  one which has incident type + year + month, and one which includes only the popular incident types.  We want to join these two sets together so that I only get the year+month data for popular incident types.  We can use the `merge()` function to do just that.

In [ ]:
popular.crimes.by.type.and.month <- merge(x = raleigh.crime.by.type.and.month, 
                                          y = popular.crimes,
                                          by = "IncidentType")

In [ ]:
boxplot(n.x ~ IncidentType, data = popular.crimes.by.type.and.month,
        xlab="Incident Type", ylab="Number of incidents per month")

We tend to see a fairly consistent number of incidents per month for each of the major types.  Larceny has a fairly wide variance, but zero incidents outside the whiskers.  Drugs has a couple of outliers and Assault has one well under the norm.  This tells us that Raleigh police tend to mark incidents fairly consistently and do not change radically from month to month.

Of course, we could say the same about the people causing these incidents...

So what about a breakdown by beat?  We don't know exactly which officers patrol a particular beat or even exactly where these beats are (though the data set does have latitude + longitude data so we might be able to back into some results there), but working from the assumption that an officer will run a beat for a while, we might see something interesting.

In [ ]:
incidents.by.beat <- raleighcrime %>%
  group_by(IncidentType, BeatID) %>%
  summarize(n = n())
popular.incidents.by.beat <- merge(x = incidents.by.beat, 
                                   y = popular.crimes,
                                   by = "IncidentType")
boxplot(n.x ~ IncidentType, data = popular.incidents.by.beat,
        xlab="Incident Type", ylab="Number of incidents by beat")

In fact, for Miscellaneous incidents, there are some huge outliers.  That might be worth further investigation.

Also worth noting is that each of these incident types are skewed toward more incidents.  I could plot the incident types as a curve and expect to see skewness, particularly for the Miscellaneous category.

Now let's focus down to a particular incident type and look at it on a per-beat basis.

In [ ]:
prostitution.by.beat <- raleighcrime %>%
                      filter(IncidentType == "PROSTITUTION") %>% 
                      group_by(BeatID) %>%
                      summarize(n = n())
plot(prostitution.by.beat)

Note that a few beats are way above the norm.  This could be because the people on those beats are more active about citing, or maybe they are working in areas known for this.  We don't have enough information to know for sure.

## Quantile-Quantile (QQ) plots

The QQ (Quantile-Quantile) plot tells how far our data is off the expected distribution.  The closer the plot is to a straight line with a 45 degree angle, the closer the distributional fit.  Let's suppose we think the number of incidents by month follows a normal distribution.

In [ ]:
raleigh.crime.by.month <- raleighcrime %>% 
  filter(IncidentYear < 2014) %>% 
  group_by(IncidentYear, IncidentMonth) %>% 
  summarize(n = n())

`qqnorm` is a version of `qqplot` which compares against a normal distribution.  `qqline` draws a line which passes through the 1st and 3rd quartiles.

In [ ]:
sdres <- sd(raleigh.crime.by.month$n)
m <- mean(raleigh.crime.by.month$n)
raleigh.crime.by.month$sd <- ((as.double(raleigh.crime.by.month$n) - m)/sdres)

qqnorm(raleigh.crime.by.month$sd)
qqline(raleigh.crime.by.month$sd)

The assumption that the number of incidents follows a normal distribution isn't a terrible assumption, but it's also not very good:  far too many months have too few incidents.

## Time Series Basics

We can look at incidents by month with a simple time-series analysis.  We will add a new variable called IncidentCount and aggregate those incident counts.

In [ ]:
raleighcrime.timeseries <- raleighcrime %>% filter(IncidentYear < 2014)
raleighcrime.timeseries$IncidentCount <- 1
raleighcrime.timeseries$Month <- as.Date(cut(raleighcrime.timeseries$IncidentDate, breaks = "month"))
ggplot(data = raleighcrime.timeseries, aes(x = Month, y = IncidentCount)) +
  stat_summary(fun.y = sum, geom = "line") +
  scale_x_date(labels = date_format("%Y-%m"), breaks = date_breaks("year"))

It seems that the number of incidents drops sharply each year...but that coincides with February.